<a href="https://colab.research.google.com/github/shaunak-badani/XAI/blob/main/RL/Reinforcement_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install -q gym-super-mario-bros==7.4.0 tensordict==0.7.2 torchrl==0.7.0
pip install -q 'numpy<2.0.0'

In [5]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os

# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

from gym import spaces

from tensordict import TensorDict
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage

In [24]:
class SimpleEnv(gym.Env):
    def __init__(self):
        super(SimpleEnv, self).__init__()

        # Action space: Space with 4 actions
        self.action_space = spaces.Discrete(4)
        print(self.action_space)

        # Observation space: 2D box of dimensions 8x8
        self.observation_space = spaces.Box(low=-4, high=4, shape = (8, 8), dtype=np.uint8)
        print(self.observation_space)

        # Initial state
        self.state = self.get_initial_random_state()

    def get_initial_random_state(self, number_of_dirt_pieces = 15):
        board = np.zeros(shape = (8, 8))
        x_indices = np.random.randint(0, 8, size = number_of_dirt_pieces)
        y_indices = np.random.randint(0, 8, size = number_of_dirt_pieces)
        board[x_indices, y_indices] = 1
        starting_position = np.random.randint(0, 8, size = 2)
        board[starting_position[0], starting_position[1]] = 2
        return board

    def reset(self):
        """Reset the environment to an initial state."""
        self.state = self.get_initial_random_state()
        return np.array(self.state, dtype=np.float32)

    def step(self, action):
        """Apply the action and return the next state, reward, done, and info."""
        # Simple logic for the environment:
        if action == 0:  # Move left
            self.state -= 1
        else:  # Move right
            self.state += 1

        # Reward function
        reward = -abs(self.state)  # Negative reward for being far from the center (0)

        # Check if the episode is done
        done = abs(self.state) > 10  # Terminate if the state goes out of bounds

        return np.array(self.state, dtype=np.float32), reward, done, {}

    def render(self, mode="human"):
        """Render the environment state."""
        print(f"Current state: {self.state}")

SimpleEnv()

Discrete(4)
Box(252, 4, (8, 8), uint8)
